<a href="https://colab.research.google.com/github/jvbristol/Convolution-Neural-Networks/blob/master/CNN_Optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=28f9d8512bd4c7fc587167f5a06867ec16e5911c52d991ef97d3835e487ab7b2
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=71caad40dd846b1ac336e8739b1a999101223c042a1c37e2c83c6f66daedbeee
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.2.0-rc2


In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()


4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)1

Epoch 1/3
1688/1688 [==============================] - 339s 201ms/step - loss: 0.4316 - accuracy: 0.8469 - val_loss: 0.3110 - val_accuracy: 0.8848
Epoch 2/3
1688/1688 [==============================] - 335s 198ms/step - loss: 0.2843 - accuracy: 0.8965 - val_loss: 0.3048 - val_accuracy: 0.8847
Epoch 3/3
1688/1688 [==============================] - 336s 199ms/step - loss: 0.2328 - accuracy: 0.9144 - val_loss: 0.2654 - val_accuracy: 0.9043


Epoch 1/3
1688/1688 [==============================] - 773s 458ms/step - loss: 0.5625 - accuracy: 0.8134 - val_loss: 0.4003 - val_accuracy: 0.8472
Epoch 2/3
1688/1688 [==============================] - 780s 462ms/step - loss: 0.4017 - accuracy: 0.8498 - val_loss: 0.3788 - val_accuracy: 0.8555
Epoch 3/3
1688/1688 [==============================] - 771s 457ms/step - loss: 0.3816 - accuracy: 0.8555 - val_loss: 0.3966 - val_accuracy: 0.8615


Epoch 1/3
1688/1688 [==============================] - 422s 250ms/step - loss: 2.3492 - accuracy: 0.1006 - val_loss: 2.3042 - val_accuracy: 0.1050
Epoch 2/3
1688/1688 [==============================] - 421s 250ms/step - loss: 2.3040 - accuracy: 0.0995 - val_loss: 2.3037 - val_accuracy: 0.0925
Epoch 3/3
1688/1688 [==============================] - 421s 249ms/step - loss: 2.3039 - accuracy: 0.1000 - val_loss: 2.3045 - val_accuracy: 0.1055


Epoch 1/3
1688/1688 [==============================] - 611s 362ms/step - loss: 0.4699 - accuracy: 0.8311 - val_loss: 0.3931 - val_accuracy: 0.8540
Epoch 2/3
1688/1688 [==============================] - 612s 363ms/step - loss: 0.3505 - accuracy: 0.8706 - val_loss: 0.3338 - val_accuracy: 0.8770
Epoch 3/3
 250/1688 [===>..........................]Buffered data was truncated after reaching the output size limit.

In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                921648    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,000,650
Trainable params: 1,000,650
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 345s 204ms/step - loss: 0.1944 - accuracy: 0.9276 - val_loss: 0.2585 - val_accuracy: 0.9037
Epoch 5/10
1688/1688 [==============================] - 336s 199ms/step - loss: 0.1608 - accuracy: 0.9404 - val_loss: 0.2625 - val_accuracy: 0.9123
Epoch 6/10
1688/1688 [==============================] - 326s 193ms/step - loss: 0.1327 - accuracy: 0.9500 - val_loss: 0.2663 - val_accuracy: 0.9115
Epoch 7/10
1688/1688 [==============================] - 342s 203ms/step - loss: 0.1079 - accuracy: 0.9589 - val_loss: 0.2860 - val_accuracy: 0.9110
Epoch 8/10
1688/1688 [==============================] - 344s 204ms/step - loss: 0.0897 - accuracy: 0.9661 - val_loss: 0.3249 - val_accuracy: 0.9107
Epoch 9/10
1688/1688 [==============================] - 347s 206ms/step - loss: 0.0748 - accuracy: 0.9714 - val_loss: 0.3509 - val_accuracy: 0.9100
Epoch 10/10
1688/1688 [==============================] - 346s 205ms/step - loss: 0.0618 - accuracy: 0.9766 - val